In [1]:
import sys
sys.path.append("..")

import time
import numpy as np
import matplotlib.pyplot as plt

import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F

import torch.nn as nn

import seaborn as sns # conda install seaborn
import pandas as pd # ^^ this will automatically install pandas

import pyro
from pyro.infer.mcmc import MCMC
import pyro.distributions as dist

from kernel.sghmc import SGHMC
from kernel.sgld import SGLD
from kernel.sgd import SGD
from kernel.sgnuts import NUTS as SGNUTS

pyro.set_rng_seed(101)

In [2]:
assert torch.cuda.is_available()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu" )

In [3]:
# Simple dataset wrapper class

class Dataset(torch.utils.data.Dataset):
    def __init__(self, data, targets):
        self.data = data
        self.targets = targets
        
    def __len__(self):
        return(len(self.data))
    
    def __getitem__(self, idx):
        return self.data[idx], self.targets[idx]

### Hyperparams

In [4]:
BATCH_SIZE = 100
NUM_EPOCHS = 800
WARMUP_EPOCHS = 50
HIDDEN_SIZE = 100

### Download CIFAR10 and setup datasets / dataloaders

In [5]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

train_dataset = datasets.CIFAR10('./data', train=True, download=True)

val_dataset = datasets.CIFAR10('./data', train=False, download=True)
    
mean = np.array([0.4912, 0.4823, 0.4468])
std = np.array([0.2470, 0.2435, 0.2616])

# scale the datasets
X_train = torch.FloatTensor((train_dataset.data / 255.0 - mean) /std)
X_train = torch.reshape(X_train, (len(X_train), 3, 32, 32))
Y_train = torch.LongTensor(train_dataset.targets)

X_val = torch.FloatTensor((val_dataset.data / 255.0 - mean) / std)
X_val = torch.reshape(X_val, (len(X_val), 3, 32, 32))
Y_val = torch.LongTensor(val_dataset.targets)

# redefine the datasets
train_dataset = Dataset(X_train, Y_train)
val_dataset = Dataset(X_val, Y_val)

# setup the dataloaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


### Define the Bayesian neural network  model

In [6]:
PyroLinear = pyro.nn.PyroModule[torch.nn.Linear]
    
class BNN(pyro.nn.PyroModule):
    
    def __init__(self, n_channels, hidden_size, output_size, prec=1., device='cpu'):
        super().__init__()
        # prec is a kwarg that should only used by SGD to set the regularization strength 
        # recall that a Guassian prior over the weights is equivalent to L2 norm regularization in the non-Bayes setting
        
        # TODO add gamma priors to precision terms
        
        self.n_channels = n_channels
        self.device = device
        
        self.conv1 = nn.Conv2d(3, n_channels, kernel_size=3, padding=1)
        #self.conv1_batchnorm = nn.BatchNorm2d(num_features=n_channels)
        self.conv2 = nn.Conv2d(n_channels, n_channels//2, kernel_size=3, padding=1)
        #self.conv2_batchnorm = nn.BatchNorm2d(num_features=n_channels//2)
        
        self.fc1 = PyroLinear(8 * 8 * n_channels // 2, hidden_size)
        
        self.fc1_w_loc = torch.zeros((hidden_size, 8 * 8 * n_channels//2), device=self.device)
        self.fc1_w_scale = torch.ones((hidden_size, 8 * 8 * n_channels//2), device=self.device) * prec
        
        self.fc1_b_loc = torch.zeros((hidden_size), device=self.device)
        self.fc1_b_scale = torch.ones((hidden_size), device=self.device) * prec
        
        self.fc1.weight = pyro.nn.PyroSample(dist.Normal(self.fc1_w_loc, self.fc1_w_scale).to_event(2))
        self.fc1.bias   = pyro.nn.PyroSample(dist.Normal(self.fc1_b_loc, self.fc1_b_scale).to_event(1))
        
        self.fc2 = PyroLinear(hidden_size, output_size)
        
        self.fc2_w_loc = torch.zeros((output_size, hidden_size), device=self.device)
        self.fc2_w_scale = torch.ones((output_size, hidden_size), device=self.device) * prec
        
        self.fc2_b_loc = torch.zeros((output_size), device=self.device)
        self.fc2_b_scale = torch.ones((output_size), device=self.device) * prec
        
        self.fc2.weight = pyro.nn.PyroSample(dist.Normal(self.fc2_w_loc, self.fc2_w_scale).to_event(2))
        self.fc2.bias   = pyro.nn.PyroSample(dist.Normal(self.fc2_b_loc, self.fc2_b_scale).to_event(1))
        
        self.log_softmax = torch.nn.LogSoftmax(dim=1)

    def forward(self, x, y=None):
        x = x.to(self.device)
        x = self.conv1(x)
        x = F.max_pool2d(torch.tanh(x), 2)
        x = self.conv2(x)
        x = F.max_pool2d(torch.tanh(x), 2)
        x = x.view(-1, 8 * 8 * self.n_channels//2)
        
        x = torch.tanh(self.fc1(x))
        x = self.fc2(x)
        x = self.log_softmax(x)# output (log) softmax probabilities of each class
        
        y = y.to(self.device)
        with pyro.plate("data", x.shape[0]):
            obs = pyro.sample("obs", dist.Categorical(logits=x), obs=y)

### Run SGHMC 

We run SGHMC to sample approximately from the posterior distribution.

In [7]:
LR = 2e-6
MOMENTUM_DECAY = 0.01
RESAMPLE_EVERY_N = 100
NUM_STEPS = 1

pyro.clear_param_store()

bnn = BNN(16, HIDDEN_SIZE, 10, device=device).to(device)

sghmc = SGHMC(bnn,
              subsample_positions=[0, 1],
              batch_size=BATCH_SIZE,
              learning_rate=LR,
              momentum_decay=MOMENTUM_DECAY,
              num_steps=NUM_STEPS,
              resample_every_n=RESAMPLE_EVERY_N,
              device=device)

sghmc_mcmc = MCMC(sghmc, num_samples=len(train_dataset)//BATCH_SIZE, warmup_steps=0, mp_context="spawn")

sghmc_test_errs = []

# full posterior predictive 
full_predictive = torch.FloatTensor(10000, 10)
full_predictive.zero_()

for epoch in range(1, 1+NUM_EPOCHS + WARMUP_EPOCHS):
    sghmc_mcmc.run(X_train, Y_train)
    
    if epoch >= WARMUP_EPOCHS:
        
        sghmc_samples = sghmc_mcmc.get_samples()
        predictive = pyro.infer.Predictive(bnn, posterior_samples=sghmc_samples)
        start = time.time()
        
        with torch.no_grad():
            epoch_predictive = None
            for x, y in val_loader:
                if epoch_predictive is None:
                    epoch_predictive = predictive(x)['obs'].to(torch.int64)
                else:
                    epoch_predictive = torch.cat((epoch_predictive, predictive(x)['obs'].to(torch.int64)), dim=1)
                    
            for sample in epoch_predictive:
                predictive_one_hot = F.one_hot(sample, num_classes=10)
                full_predictive = full_predictive + predictive_one_hot
                
            full_y_hat = torch.argmax(full_predictive, dim=1)
            total = Y_val.shape[0]
            correct = int((full_y_hat == Y_val).sum())
            
        end = time.time()
        
        sghmc_test_errs.append(1.0 - correct/total)

        print("Epoch [{}/{}] test accuracy: {:.4f} time: {:.2f}".format(epoch-WARMUP_EPOCHS, NUM_EPOCHS, correct/total, end - start))

Sample:  26%|█████████████████▏                                               | 132/500 [00:02, 40.88it/s, lr=2.00e-06]

KeyboardInterrupt: 